## モノイド
モノイドとは型クラス。値を二項演算子で結合できるような型を表す。
## newtypeキーワード
既存の型から新しい型を作る。既存の型をくるんで新しい型に見せる。元の型と同一の内部表現で扱うのでdataより高速に作動する。
```
newtype ZipList a = ZipList {getZipList :: [a]}
```

In [2]:
import Control.Applicative
getZipList $ ZipList [(+1),(*100),(*5)] <*> ZipList [1,2,3]

[2,200,15]

In [6]:
newtype CharList = CharList {getCharList :: [Char]} deriving(Eq,Show)
CharList "this will be shown!"
:t CharList

CharList {getCharList = "this will be shown!"}

CharList :: [Char] -> CharList

In [7]:
newtype Pair b a = Pair { getPair :: (a,b)}
instance Functor (Pair c) where
    fmap f (Pair (x,y)) = Pair (f x,y) --パターンマッチも使える
getPair $ fmap reverse (Pair ("london",3))

("nodnol",3)

## 復習
- type:型シノニムを作る。既存の型に別名をつける。型シグネチャを読みやすくする目的で使われる。
- newtype:既存の型を包んで新しい型を作る。型クラスのインスタンスを作りやすくする目的で使われる。出来上がる型はtypeと違って別物になる。値コンストラクタ・フィールドが一つだけ。
- data:自作の新しいデータ型
## Monoid型クラス
結合的な二項演算子とその演算に関する単位元からなる構造。
 - 関数は引数を2つとる
 - 2つの引数及び返り値の型は全て等しい
 - 2引数関数を施して相手を変えないような特殊な値が存在する(単位元的な)
 - 結合的(関数を挟む順序を変えても結果が変わらない)
 ```
 class Monoid m where
     mempty :: m
     mappend :: m->m->m --モノイド固有の二項演算
     mconcat :: [m] -> m --リストをとって単一の値に　
     mconcat = foldr mappend mempty --デフォルト実装
 ```
 ProductとSum(Num), AnyとAll(Bool), Orderingなど

In [10]:
lengthCompare x y = (length x `compare` length y) `mappend` (x `compare` y)
lengthCompare "zen" "ants"
lengthCompare "zen" "ant"

LT

GT

## 畳み込み


In [12]:
import qualified Data.Foldable as F
F.foldr (||) False (Just True)

True

## foldMap
「foldableにしたいコンテナの中身の型の値をとってmonoidを返す関数」を第１引数、構造体を第2引数、モノイドを結果として返す関数。まず第2引数の構造体に第1引数の関数をmapして構造体の中身をごっそりモノイド値にかえ、それらモノイド値をmappendして単一のモノイド値へと結合。